In [0]:
from sklearn import metrics
import pandas as pd
import numpy as np
import glob
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
import time
import collections
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import defaultdict


In [3]:
from google.colab import drive
drive.mount('/gdrive/')
%cd /gdrive/My\ Drive/artigo_plantas
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/
/gdrive/My Drive/artigo_plantas
resnet152v2_features.csv  resnet50_features.csv


In [0]:
#Variáveis do Dataset de Resultados
arquivo_csv = []
classifier = []
accuracy = []
std_accuracy = []

# Metrics Macro
precision_macro = []
std_precision_macro = []
recall_macro = []
std_recall_macro = []
f1_macro = []
std_f1_macro = []

# Metrics Micro
precision_micro = []
std_precision_micro = []
recall_micro = []
std_recall_micro = []
f1_micro = []
std_f1_micro = []

# Metrics Weighted
precision_weighted = []
std_precision_weighted = []
recall_weighted = []
std_recall_weighted = []
f1_weighted = []
std_f1_weighted = []

# Informações do PCA
n_samples = []
princ_compo = []
variancia = []


In [0]:
def multi_clf_metrics(arq_csv):

	# Cria X (features) e y (labels)
  especies = pd.read_csv(arq_csv, low_memory=False)
	
  if arq_csv == 'resnet50_features.csv' or arq_csv == 'resnet152v2_features.csv':
    X = np.array(especies.iloc[0:,0:2048])
    y = np.array(especies.iloc[0:,2048])		
  # Executa o PCA
  pca = PCA(.90)
  principal_components = pca.fit_transform(X)

  classifiers = {'SVC - (linear, ovr)': OneVsRestClassifier(svm.SVC(kernel='linear', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=500,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovr', random_state=None), n_jobs=-1),
					'SVC - (linear, ovo)': svm.SVC(kernel='linear', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=500,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovo', random_state=None),
					'SVC - (polynomial, ovr)': OneVsRestClassifier(svm.SVC(kernel='poly', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=500,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovr', random_state=None), n_jobs=-1),
					'SVC - (polynomial, ovo)': svm.SVC(kernel='poly', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=500,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovo', random_state=None),
					'SVC - (sigmoid, ovr)': OneVsRestClassifier(svm.SVC(kernel='sigmoid', degree=3,coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=500,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovr', random_state=None), n_jobs=-1),
					'SVC - (sigmoid, ovo)': svm.SVC(kernel='sigmoid', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=500,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovo', random_state=None)}

  scoring = ['f1_micro']
  # Parametros para rodar com o grid search
  parameters = {"estimator__C" : [0.01, 0.1, 1, 10, 100, 1000], 'estimator__gamma':[0.001, 0.01, 0.1, 1]}
  results = []
  for name_clf, clf in classifiers.items():
    print(f'Executando classificador \033[1;31m{name_clf}\033[m')
    scores = GridSearchCV(clf, parameters,cv=5, scoring=scoring, n_jobs=-1, refit='f1_micro', return_train_score=True)
    result = scores.fit(principal_components,y)
    results.append(result)
    print('Melhor configuração: '+result.best_params_)
  return result

In [0]:
result_resnet50 = multi_clf_metrics('resnet50_features.csv')

Executando classificador SVC - (linear, ovr)
